# Requete avec les paramètres du jeu de test

In [50]:
#cells pour préparer les requêtes
import urllib.parse

domain = "51.91.251.0"
port = 3000
host = f"http://{domain}:{port}"
path = lambda x: urllib.parse.urljoin(host, x)

In [51]:
import requests

user_id = '545115b6-c8cf-4dbd-9e1c-a8270ea9bd66'
name = 'AlorsLaZoneCaDitQuoi'
r = requests.post(path(f'avatars/{user_id}/{name}'))
r

<Response [200]>

In [52]:
r.json()

{'name': 'AlorsLaZoneCaDitQuoi',
 'created_at': '2023-01-04T16:09:04.957735',
 'user_id': '545115b6-c8cf-4dbd-9e1c-a8270ea9bd66',
 'id': 334159}

In [53]:
r = requests.get(path(f"avatars/{user_id}"))
for avatar in r.json():
    print(avatar['id'], avatar['name'])

6054 max-first-avatar
6055 max-second-avatar
111612 max-5th-avatar
71508 max-third-avatar
71509 max-4th-avatar
303615 max-f-avatar
303616 test
303617 a
334159 AlorsLaZoneCaDitQuoi
334156 Flo
334157 FloDesLoges
334158 AlorsLaZone


In [54]:
import numpy as np
import matplotlib.pyplot as plt
import plotly
import pandas as pd

In [55]:
#importation du jeu test
hotels = pd.read_csv('features_hotels.csv', index_col=['hotel_id', 'city'])
Xtest = pd.read_csv("test_set.csv",index_col =0)
Xtest = Xtest.join(hotels, on=['hotel_id', 'city'])
Xtest

,order_requests,city,date,language,mobile,avatar_id,hotel_id,stock,group,brand,parking,pool,children_policy
index,,,,,,,,,,,,,
0,1,vilnius,21,romanian,0,1,161,46,Boss Western,J.Halliday Inn,1,0,0
1,1,vilnius,21,romanian,0,1,187,32,Accar Hotels,Marcure,1,1,0
2,1,vilnius,21,romanian,0,1,279,12,Independant,Independant,1,0,0
3,1,vilnius,21,romanian,0,1,395,10,Accar Hotels,Ibas,0,0,0
4,1,vilnius,21,romanian,0,1,488,42,Accar Hotels,Safitel,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6639,843,rome,5,irish,0,794,987,1,Accar Hotels,Ibas,1,0,0
6640,844,vienna,1,irish,1,794,26,1,Accar Hotels,Marcure,1,0,0
6641,844,vienna,1,irish,1,794,263,0,Boss Western,Boss Western,1,0,0


In [56]:
#transformation des colonnes intéressantes en liste de dictionnaires, ordonné en dates décroissantes
Xrequete = Xtest[["city","date","language","mobile"]]
Xrequete = Xrequete.drop_duplicates()
Xrequete = Xrequete.sort_values(by="date",ascending = False)
Xrequete["avatar_name"]="test"
Xrequete =Xrequete[["avatar_name","language","city","date","mobile"]]
Requete = Xrequete.to_dict(orient='records')

In [57]:
#Requetes
R = []
for params in Requete:
    r = requests.get(path(f"pricing/{user_id}"), params=params)
    r.json()
    R+= [r]

In [58]:
for r in R:
    print(r.json())

{'detail': 'A pricing request for this avatar already exists for a sooner date'}
{'detail': 'A pricing request for this avatar already exists for a sooner date'}
{'detail': 'A pricing request for this avatar already exists for a sooner date'}
{'detail': 'A pricing request for this avatar already exists for a sooner date'}
{'detail': 'A pricing request for this avatar already exists for a sooner date'}
{'detail': 'A pricing request for this avatar already exists for a sooner date'}
{'detail': 'A pricing request for this avatar already exists for a sooner date'}
{'detail': 'A pricing request for this avatar already exists for a sooner date'}
{'detail': 'A pricing request for this avatar already exists for a sooner date'}
{'detail': 'A pricing request for this avatar already exists for a sooner date'}
{'detail': 'A pricing request for this avatar already exists for a sooner date'}
{'detail': 'A pricing request for this avatar already exists for a sooner date'}
{'detail': 'A pricing reques

In [59]:
#Creation du tableau avec les résultats
pricing_requests = []

for r in R:
    if(r.json() != {'detail': 'A pricing request for this avatar already exists for a sooner date'}):
        pricing_requests.append(
            pd.DataFrame(r.json()['prices']).assign(**r.json()['request'])
    )

pricing_requests = pd.concat(pricing_requests)

In [60]:
# ajouts des autres features
pricing_requests = pricing_requests.join(hotels, on=['hotel_id', 'city'])

In [61]:
pricing_requests

,hotel_id,price,stock,city,date,language,mobile,avatar_id,group,brand,parking,pool,children_policy
0,141,97,0,sofia,0,french,0,303616,Boss Western,J.Halliday Inn,1,0,0
1,270,131,1,sofia,0,french,0,303616,Morriott International,Morriot,0,0,0
2,201,105,1,sofia,0,french,0,303616,Chillton Worldwide,Tripletree,0,0,0
3,578,129,11,sofia,0,french,0,303616,Yin Yang,Royal Lotus,0,0,0
4,107,52,0,sofia,0,french,0,303616,Yin Yang,8 Premium,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,839,235,0,vienna,0,swedish,0,303616,Yin Yang,Royal Lotus,0,0,0
98,730,235,0,vienna,0,swedish,0,303616,Morriott International,Morriot,0,0,1
99,343,90,0,vienna,0,swedish,0,303616,Independant,Independant,0,0,0
100,928,93,0,vienna,0,swedish,0,303616,Accar Hotels,Ibas,0,0,0


In [62]:
#Le jeu de test de Noel :'( vivement la fin du GMM ...
pricing_requests.to_csv('Xtrain4_1ter.csv')